In [25]:
from google.colab import drive
drive.mount('/content/drive')
!pip install  alphabet_detector 
!pip install pyarabic
import nltk 
nltk.download('stopwords')
nltk.download('punkt')

import pandas as pd
# Load the dataset into a pandas dataframe.
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/datasetfinal.xlsx")

from alphabet_detector import AlphabetDetector
import re
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.stem.isri import ISRIStemmer

#remove any othe language letters
def keep_only_arabic(text):
    ad = AlphabetDetector()
    clean_lines = list()
    for line in text.splitlines():
        clean_line = list()
        for word in line.split():
            if len(word) > 1:
                if ad.is_arabic(word):
                    if word.isalpha():
                        clean_line.append(word)
        clean_lines.append(' '.join(clean_line))
    return '\n'.join(clean_lines)

# remove empty lines
def remove_empty_lines(text):
    lines = [s.rstrip() for s in text.split("\n") if s.rstrip()]
    return '\n'.join(lines)

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
import pyarabic.araby as araby
def normalizeArabic(text):
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    text = re.sub(r'(.)\1+', r"\1", text) # Remove longation
    return araby.strip_tashkeel(text)
    
#delete everything except text 
def char_rmvl(sentence):
  sen_split = sentence.split()
  for idx1,word in enumerate(sen_split):       
             sen_split[idx1] = re.sub('[^a-zا-ي]', '', word)       
  sentence = ' '.join(sen_split)    
  return sentence 
stops = set(stopwords.words("arabic"))
stop_word_comp = {"،","آض","آمينَ","آه","آهاً","آي","أ","أب","أجل","أجمع","أخ","أخذ","أصبح",
                  "أضحى","أقبل","أقل","أكثر","ألا","أم","أما","أمامك","أمامكَ","أمسى","أمّا","أن","أنا","أنت","","أنتما","أنتن","أنتِ",
                  "أنشأ","أنّى","أو","أوشك","أولئك","أولئكم","أولاء","أولالك","أوّهْ","أي","أيا","أين","أينما","أيّ","أَنَّ","أََيُّ","أُفٍّ","إذ","إذا",
                  "إذاً","إذما","إذن","إلى","إليكم","إليكما","إليكنّ","إليكَ","إلَيْكَ","إلّا","إمّا","إن","إنّما","إي","إياك","إياكم","إياكما","إياكن",
                  "إيانا","إياه","إياها","إياهم","إياهما","إياهن","إياي","إيهٍ","إِنَّ","ا","ابتدأ","اثر","اجل","احد","اخرى","اخلولق","اذا","اربعة",
                  "ارتدّ","استحال","اطار","اعادة","اعلنت","اف","اكثر","اكد","الألاء","الألى","الا","الاخيرة","الان","الاول","الاولى","التى","التي",
                  "الثاني","الثانية","الذاتي","الذى","الذي","الذين","السابق","الف","اللائي","اللاتي","اللتان","اللتيا","اللتين","اللذان","اللذين","اللواتي",
                  "الماضي","المقبل","الوقت","الى","اليوم","اما","امام","امس","ان","انبرى","انقلب","انه","انها","او","اول","اي","ايار","ايام",
                  "ايضا","ب","بات","باسم","بان","بخٍ","برس","بسبب","بسّ","بشكل","بضع","بطآن","بعد","بعض","بك","بكم","بكما","بكن",
                  "بل","بلى","بما","بماذا","بمن","بن","بنا","به","بها","بي","بيد","بين","بَسْ","بَلْهَ","بِئْسَ","تانِ","تانِك","تبدّل","تجاه","تحوّل",
                  "تلقاء","تلك","تلكم","تلكما","تم","تينك","تَيْنِ","تِه","تِي","ثلاثة","ثم","ثمّ","ثمّة","ثُمَّ","جعل","جلل","جميع","جير","حار","حاشا",
                  "حاليا","حاي","حتى","حرى","حسب","حم","حوالى","حول","حيث","حيثما","حين","حيَّ","حَبَّذَا","حَتَّى","حَذارِ","خلا","خلال","دون",
                  "دونك","ذا","ذات","ذاك","ذانك","ذانِ","ذلك","ذلكم","ذلكما","ذلكن","ذو","ذوا","ذواتا","ذواتي","ذيت","ذينك","ذَيْنِ","ذِه","ذِي",
                  "راح","رجع","رويدك","ريث","رُبَّ","زيارة","سبحان","سرعان","سنة","سنوات","سوف","سوى","سَاءَ","سَاءَمَا","شبه","شخصا","شرع",
                  "شَتَّانَ","صار","صباح","صفر","صهٍ","صهْ","ضد","ضمن","طاق","طالما","طفق","طَق","ظلّ","عاد","عام","عاما","عامة","عدا",
                  "عدة","عدد","عدم","عسى","عشر","عشرة","علق","على","عليك","عليه","عليها","علًّ","عن","عند","عندما","عوض","عين","عَدَسْ",
                  "عَمَّا","غدا","غير","ـ","ف","فان","فلان","فو","فى","في","فيم","فيما","فيه","فيها","قال","قام","قبل","قد","قطّ","قلما","قوة","كأنّما",
                  "كأين","كأيّ","كأيّن","كاد","كان","كانت","كذا","كذلك","كرب","كل","كلا","كلاهما","كلتا","كلم","كليكما","كليهما","كلّما","كلَّا","كم",
                  "كما","كي","كيت","كيف","كيفما","كَأَنَّ","كِخ","لئن","لا","لات","لاسيما","لدن","لدى","لعمر","لقاء","لك","لكم","لكما","لكن",
                  "لكنَّما","لكي","لكيلا","للامم","لم","لما","لمّا","لن","لنا","له","لها","لو","لوكالة","لولا","لوما","لي","لَسْتَ","لَسْتُ","لَسْتُم","لَسْتُمَا",
                  "لَسْتُنَّ","لَسْتِ","لَسْنَ","لَعَلَّ","لَكِنَّ","لَيْتَ","لَيْسَ","لَيْسَا","لَيْسَتَا","لَيْسَتْ","لَيْسُوا","لَِسْنَا","ما","ماانفك","مابرح","مادام","ماذا","مازال",
                  "مافتئ","مايو","متى","مثل","مذ","مساء","مع","معاذ","مقابل","مكانكم","مكانكما","مكانكنّ","مكانَك","مليار","مليون","مما","ممن",
                  "من","منذ","منها","مه","مهما","مَنْ","مِن","نحن","نحو","نعم","نفس","نفسه","نهاية","نَخْ","نِعِمّا","نِعْمَ","ها","هاؤم","هاكَ","هاهنا",
                  "هبّ","هذا","هذه","هكذا","هل","هلمَّ","هلّا","هم","هما","هن","هنا","هناك","هنالك","هو","هي","هيا","هيت","هيّا","هَؤلاء",
                  "هَاتانِ","هَاتَيْنِ","هَاتِه","هَاتِي","هَجْ","هَذا","هَذانِ","هَذَيْنِ","هَذِه","هَذِي","هَيْهَاتَ","و","و6","وا","واحد","واضاف","واضافت",
                  "واكد","وان","واهاً","واوضح","وراءَك","وفي","وقال","وقالت","وقد","وقف","وكان","وكانت","ولا","ولم","ومن","مَن","وهو",
                  "وهي","ويكأنّ","وَيْ","وُشْكَانََ","يكون","يمكن","يوم","ّأيّان","أنتم" , 'إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 
                  'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 
                    'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى',
                    'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن',
                    'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا',
                    'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 
                    'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 
                    'فيها', 'قد', 'كأن', 'كأنما', 'كأي', 'كأين', 'كذا', 'كذلك', 'كل', 'كلا', 'كلاهما', 'كلتا', 'كلما', 'كليكما', 'كليهما', 'كم', 'كم', 'كما', 'كي', 'كيت', 'كيف', 'كيفما', 'لا',
                    'لاسيما', 'لدى', 'لست', 'لستم', 'لستما', 'لستن', 'لسن', 'لسنا', 'لعل', 'لك', 'لكم', 'لكما', 'لكن', 'لكنما', 'لكي', 'لكيلا', 'لم', 'لما', 'لن', 'لنا', 'له', 'لها', 'لهم', 'لهما', 
                    'لهن', 'لو', 'لولا', 'لوما', 'لي', 'لئن', 'ليت', 'ليس', 'ليسا', 'ليست', 'ليستا', 'ليسوا', 'ما', 'ماذا', 'متى', 'مذ', 'مع', 'مما', 'ممن', 'من', 'منه', 'منها', 'منذ', 'مه', 'مهما',
                    'نحن', 'نحو', 'نعم', 'ها', 'هاتان', 'هاته', 'هاتي', 'هاتين', 'هاك', 'هاهنا', 'هذا', 'هذان', 'هذه', 'هذي', 'هذين', 'هكذا', 'هل', 'هلا', 'هم', 'هما', 'هن', 'هنا', 'هناك', 'هنالك',
                    'هو', 'هؤلاء', 'هي', 'هيا', 'هيت', 'هيهات', 'والذي', 'والذين', 'وإذ', 'وإذا', 'وإن', 'ولا', 'ولكن', 'ولو', 'وما', 'ومن', 'وهو', 'يا', 'يمالى', 'قل', 'كثر', 'ألي', 'ليك', 'ليكم', 
                    'نتيا', 'نتوما', 'رانا' ,'بصح', 'هوما', 'وين', 'أمبعد', 'أومبعد', 'شوية', 'شويا', 'وش', 'واش', 'بوش', 'بواش','و'}
def remove_stop_words(text):
    zen = TextBlob(text)
    words = zen.words
    return " ".join([w for w in words if not w in stops and not w in stop_word_comp and len(w) >= 2])


def all_togther(text):
  text = keep_only_arabic(text)
  text = remove_empty_lines(text)
  text = remove_emoji(text)
  text = normalizeArabic(text)
  text = char_rmvl(text)
  text = text.lower()
  text = remove_stop_words(text)
  text = ISRIStemmer().suf32(text)
  return text


df['comments'] = df['comments'].apply(lambda x:all_togther(x))
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.dropna(subset = ["comments"], inplace=True)
#print(remove_emoji('🌏.حتي الكويت عطلة رسميه 🌷 المولد 👑❤👑🤲🤲🇲🇦🇲🇦🇲🇦🇲🇦النبوي ✌ ماذا⌨↪•⏱⏳⏰⏲#⃣ ⏪ 📺يقصد 💩صطيف ♥️🤲  ⁦فركاشه?Fakio👑👑+🇮🇱🇮🇱🐪🐪🐪🐪🐪🐪🐪👉')) #we didn't catch some emoji
#print(normalizeArabic('مَاذَااَتَخِذُ مِنٛ اَجٛرَررررراءَتٛ؟')) #done 
#print(char_rmvl('مَاذَااَتَخِذُ05550 #$%&()*+,،-./:;<=>؟?@[\]^_`{|}مِنٛ اَجٛرَررررراءَتٛ  👑❤👑🤲🤲🇲🇦🇲🇦🇲🇦🇲🇦النبوي ✌ ماذا⌨↪•⏱⏳⏰⏲#⃣ ⏪ 📺؟')) #done 
#print(remove_stop_words('دير حل رانا كرهنا من الوعود الكاذبة والهدرة بلاش')) #done 


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


downloading Fastext 

In [26]:

#Download FastText Embeddings (Vectors not the model)
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.vec.gz
!gzip -q cc.ar.300.vec.gz -d .

#%%

#Loading FastText Embeddings.
from tqdm import tqdm
import codecs
import numpy as np

"""Loading FastText Embeddings"""
print("Loading FastText Embeddings...")
FastText = {}
f = codecs.open('/content/cc.ar.300.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    FastText[word] = coefs
f.close()

#%%


--2021-08-08 15:47:36--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1272365870 (1.2G) [binary/octet-stream]
Saving to: ‘cc.ar.300.vec.gz’

cc.ar.300.vec.gz    100%[===================>]   1.18G  21.8MB/s    in 56s     

2021-08-08 15:48:34 (21.5 MB/s) - ‘cc.ar.300.vec.gz’ saved [1272365870/1272365870]

gzip: cc.ar.300.vec already exists; do you wish to overwrite (y or n)? y
y



934it [00:00, 9333.31it/s]

Loading FastText Embeddings...


2000001it [03:46, 8843.05it/s]


fasttext_embedding matrix

In [27]:

#Tokenization and Sentence Encoding

X_train_1 = df['comments']

#Input_ids and Attention masks.
X_train = []

#Creating Embedding Matrix for fasttext embeddings
from keras.preprocessing import sequence
from keras.preprocessing import text
tokenizer = text.Tokenizer(num_words=30000)
tokenizer.fit_on_texts(X_train_1)

train_texts = tokenizer.texts_to_sequences(X_train_1)
X_train_ft = sequence.pad_sequences(train_texts, maxlen=128)

print("Dictionary: "+str(len(tokenizer.word_index)))
embedding_matrix = np.zeros((30000, 300))
for word, index in tokenizer.word_index.items():
    if index > 30000 - 1:
        break
    else:
        embedding_vector = FastText.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector



Dictionary: 39137


spliting 

In [28]:
from sklearn.model_selection import train_test_split
y = df['hate '].map({'yes':1,'no':0 })
##Splitting fastText encoded data for training DL algorithms based on Bi-LSTMs and Bi-GRUs.
## train data ==> (X_train_ft, y_train)
## Validation data ==> (X_val_ft, y_val)
## Test data ==> (X_test_ft, y_test)
X_train, X_test, y_train, y_test = train_test_split(X_train_ft,y)


fasttext with LSTM

In [29]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import warnings
warnings.filterwarnings('ignore')

model = Sequential()
## embeddings layer (fasttext embedding 300 dimension size)
model.add(Embedding(30000, 300, input_length=128, weights=[embedding_matrix], trainable=True))

model.add(LSTM(units=128))
#model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.summary()

model.fit(X_train, y_train, validation_split=0.3, epochs=4)

y_pred = (model.predict(X_test) >= 0.5)

accuracy_score(y_test, y_pred)

print(classification_report(y_test, y_pred))

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 128, 300)          9000000   
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)               219648    
_________________________________________________________________
dropout_16 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 129       
Total params: 9,219,777
Trainable params: 9,219,777
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4
152/152 [==============================] - 65s 412ms/step - loss: 0.6485 - acc: 0.6307 - val_loss: 0.6112 - val_acc: 0.6684
Epoch 2/4
152/152 [==============================] - 62s 409ms/step - loss: 0.3427 - acc: 0.8566 - val_loss

fasttext with GRU 

In [30]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import warnings
warnings.filterwarnings('ignore')

model = Sequential()
## embeddings layer (fasttext embedding 300 dimension size)
model.add(Embedding(30000, 300, input_length=128, weights=[embedding_matrix], trainable=True))

model.add(GRU(units=128))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.summary()

model.fit(X_train, y_train, validation_split=0.3, epochs=4)

y_pred = (model.predict(X_test) >= 0.5)

accuracy_score(y_test, y_pred)

print(classification_report(y_test, y_pred))

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 128, 300)          9000000   
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               165120    
_________________________________________________________________
dropout_17 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 129       
Total params: 9,165,249
Trainable params: 9,165,249
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4
152/152 [==============================] - 55s 353ms/step - loss: 0.6491 - acc: 0.6098 - val_loss: 0.6192 - val_acc: 0.6583
Epoch 2/4
152/152 [==============================] - 47s 311ms/step - loss: 0.3388 - acc: 0.8628 - val_loss

fasttext with Bi-LSTM

In [31]:
import tensorflow as tf
import numpy as np
tf.__version__
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import warnings

model = Sequential()
model.add(tf.keras.layers.Embedding(30000, 300, input_length=128, weights=[embedding_matrix], trainable=True))

model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])


model.summary()

model.fit(X_train, y_train, validation_split=0.3, epochs=4)

y_pred = (model.predict(X_test) >= 0.5)

accuracy_score(y_test, y_pred)

print(classification_report(y_test, y_pred))


Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 128, 300)          9000000   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 256)               439296    
_________________________________________________________________
dropout_18 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 257       
Total params: 9,439,553
Trainable params: 9,439,553
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4
152/152 [==============================] - 109s 695ms/step - loss: 0.6592 - binary_accuracy: 0.6063 - val_loss: 0.6057 - val_binary_accuracy: 0.6684
Epoch 2/4
152/152 [==============================] - 106s 695ms/step - loss: 0.355

CNN with fasttext

In [32]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import warnings
warnings.filterwarnings('ignore')

model = Sequential()
## embeddings layer (fasttext embedding 300 dimension size)
model.add(Embedding(30000, 300, input_length=128, weights=[embedding_matrix], trainable=True))
model.add(Conv1D(activation='relu', filters=4, kernel_size=4))

model.add(Dropout(0.5))

model.add(MaxPool1D())


model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.summary()

model.fit(X_train, y_train, validation_split=0.3, epochs=4)

y_pred = (model.predict(X_test) >= 0.5)

accuracy_score(y_test, y_pred)

print(classification_report(y_test, y_pred))


Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 128, 300)          9000000   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 125, 4)            4804      
_________________________________________________________________
dropout_19 (Dropout)         (None, 125, 4)            0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 62, 4)             0         
_________________________________________________________________
dense_18 (Dense)             (None, 62, 1)             5         
Total params: 9,004,809
Trainable params: 9,004,809
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4
152/152 [==============================] - 21s 131ms/step - loss: 0.6871 - acc: 0.5585 - val_loss: 

ValueError: ignored